In [2]:
import requests
import json
from configparser import ConfigParser, ExtendedInterpolation
from urllib.parse import urlparse
import time
import os

# Configura o certificado Petrobras para a lib requests
PTB_CERT_PATH = os.path.join(os.path.abspath(''), '../petrobras-ca-root.pem')
os.environ['REQUESTS_CA_BUNDLE'] = PTB_CERT_PATH

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../config.ini', 'UTF-8')

base_url = "https://apid.petrobras.com.br/ia/visao/v1/azure-ai-vision"

default_headers = {
    "Content-Type": "application/octet-stream",
    "apikey": config['HUB']['apikey_modelos_visao'],
}

def consultar_resultado(url, wait_time_seconds=3, max_retries=3):

    parsed_url = urlparse(url)
    for i in range(max_retries):
        time.sleep(wait_time_seconds)

        response = requests.get(
            f'{base_url}{parsed_url.path}',
            headers=default_headers
        )
        
        body = response.json()
        if body.get('status', '') == 'succeeded':
            return body
        else:
            print(f'Consulta de resultado retornou: {json.dumps(body, indent=4)}')



with open("presentation.png", "rb") as f:
    image_data = f.read()

response = requests.post(
    f"{base_url}/vision/v3.2/read/analyze",
    headers=default_headers,
    data=image_data,
)

if response.status_code != 202:
    print(f'Erro na requisição ({response.status_code}): {json.dumps(response.json(), indent=4)}')
else:
    
    resultado = consultar_resultado(response.headers['Operation-Location'])
    print(f'Resultado: {json.dumps(resultado, indent=4)}')



Resultado: {
    "status": "succeeded",
    "createdDateTime": "2024-04-17T12:59:53Z",
    "lastUpdatedDateTime": "2024-04-17T12:59:54Z",
    "analyzeResult": {
        "version": "3.2.0",
        "modelVersion": "2022-04-30",
        "readResults": [
            {
                "page": 1,
                "angle": 0,
                "width": 1038,
                "height": 692,
                "unit": "pixel",
                "lines": [
                    {
                        "boundingBox": [
                            130,
                            129,
                            215,
                            130,
                            215,
                            149,
                            130,
                            148
                        ],
                        "text": "9:35 AM",
                        "appearance": {
                            "style": {
                                "name": "other",
                                "